# `CLOUD.jl` - 1D linear advection-diffusion example

In [1]:
using CLOUD, OrdinaryDiffEq

Define the physical problem

In [2]:
a = 1.0  # wave speed
b = 1.0e-3  # diffusion coefficient

A = 1.0  # amplitude
L = 1.0  # domain length
σ = 0.1  # width of Gaussian pulse
x₀ = 0.5  # centre of Gaussian pulse
T = 1.0  # end time for one period

initial_data = InitialDataGaussian(A,σ,x₀)
conservation_law = LinearAdvectionDiffusionEquation(a,b);

Set discretization parameters

In [3]:
M = 4
p = 6
form = WeakConservationForm(StandardMapping(),LaxFriedrichsNumericalFlux())
strategy = Lazy()

ode_algorithm = CarpenterKennedy2N54()
dt = 0.1*(L/M)/(a*(2*p+1));
write_interval = floor(Int, T/(dt*10));

Set up a `DGSEM` scheme with LGL quadrature of degree `p` with `M` elements

In [4]:
reference_approximation =ReferenceApproximation(
    DGSEM(p), Line(), volume_quadrature_rule=LGLQuadrature())

mesh = uniform_periodic_mesh(
    reference_approximation.reference_element, (0.0,L), M)

spatial_discretization = SpatialDiscretization(mesh, 
    reference_approximation)

results_path = save_project(conservation_law,
     spatial_discretization, initial_data, form, (0.0, T), Lazy(),
     "results/advection_diffusion_1d_dgsem/", overwrite=true, clear=true);

Run the solver and plot the solution

In [5]:
ode_problem = semidiscretize(conservation_law,
    spatial_discretization,
    initial_data, 
    form,
    (0.0, T),
    strategy)
save_solution(ode_problem.u0, 0.0, results_path, 0)
CLOUD_reset_timer()
sol = solve(ode_problem, ode_algorithm, adaptive=false, dt=dt,
    save_everystep=false, callback=save_callback(results_path, write_interval))
CLOUD_print_timer()

save_solution(last(sol.u), last(sol.t), results_path, "final")
plotter = Plotter(spatial_discretization, results_path)
visualize(last(sol.u), plotter, "approx.pdf");

 ────────────────────────────────────────────────────────────────────────────────
                                        Time                    Allocations      
                               ───────────────────────   ────────────────────────
       Tot / % measured:            414ms / 226.9%           57.6MiB / 473.5%    

 Section               ncalls     time    %tot     avg     alloc    %tot      avg
 ────────────────────────────────────────────────────────────────────────────────
 auxiliary variable     10.4k    575ms   61.2%  55.2μs    180MiB   66.2%  17.8KiB
   apply operators      10.4k    445ms   47.4%  42.8μs    135MiB   49.5%  13.3KiB
     facet terms        10.4k    225ms   23.9%  21.6μs   70.3MiB   25.8%  6.92KiB
     volume terms       10.4k    135ms   14.4%  13.0μs   45.4MiB   16.7%  4.47KiB
     mass matrix solve  10.4k   4.76ms    0.5%   458ns   3.33MiB    1.2%     336B
   eval solution        10.4k    114ms   12.2%  11.0μs   37.7MiB   13.8%  3.71KiB
   eval num tra

Calculate the integral L<sup>2</sup> error norm and conservation/energy error

In [6]:
error_analysis = ErrorAnalysis(results_path, conservation_law, 
    spatial_discretization)
conservation_analysis = PrimaryConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)
energy_analysis = EnergyConservationAnalysis(results_path, 
    conservation_law, spatial_discretization)

println("L2 error:")
println(analyze(error_analysis, last(sol.u), initial_data)...)
println("Conservation (initial/final/diff):")
println(analyze(conservation_analysis)...)
println("Energy (initial/final/diff):")
println(analyze(energy_analysis)...)

L2 error:
0.03248649764825698
Conservation (initial/final/diff):
[0.2506626716404719][0.2506626716404735][1.609823385706477e-15]
Energy (initial/final/diff):
[0.0886231795493749][0.08090045567521623][-0.007722723874158677]
